# Project 3 - Part 1 

-Name: Tyler Schelling

-Date Started: 12/13/2022

---

For this project, you have been hired to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately, you will use this database to analyze what makes a movie successful and will provide recommendations to the stakeholder on how to make a successful movie. 

---

## Import Libraries

In [8]:
import os
import pandas as pd
import numpy as np

## Downloading the Files

In [9]:
basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'

In [10]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [11]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [12]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1925
1,tt0000002,5.8,261
2,tt0000003,6.5,1741
3,tt0000004,5.6,176
4,tt0000005,6.2,2554


In [13]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


## Filtering/Cleaning

In [14]:
#Replace null values ("\N") with np.nan across all 3 tables
basics = basics.replace({'\\N':np.nan})
ratings = ratings.replace({'\\N':np.nan})
akas = akas.replace({'\\N':np.nan})

In [15]:
#Eliminate movies that are null for runtimeMinutes
basics = basics[basics['runtimeMinutes'].notna()]

In [16]:
#Keep only titleType of 'Movie'
basics = basics[basics['titleType'] == 'movie']

In [17]:
#Eliminate movies that are null or contain documentary as their genre.
basics = basics[(basics['genres'].notna())]
basics = basics[~basics['genres'].str.contains('documentary',case=False)]

In [18]:
#Keep startYear 2000-2022
basics = basics[basics['startYear'].notna()]
basics['startYear'] = basics['startYear'].astype('int64')

year_filter = (basics['startYear'] > 2000) & (basics['startYear'] < 2023)
basics = basics[year_filter]

In [19]:
#Keep only movies made in the US
akas = akas[akas['region'] == 'US']
us_filter_basics = basics['tconst'].isin(akas['titleId'])
us_filter_ratings = ratings['tconst'].isin(akas['titleId'])

#Filter the basics DF for US movies
basics = basics[us_filter_basics]
#Filter the ratings DF for US movies
ratings = ratings[us_filter_ratings]

## Saving the Filtered Tables to the Repository

In [20]:
#Create Data folder in Repository to store CSV files
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [21]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

## Open Filtered Tables 

In [22]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [23]:
# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1925
1,tt0000002,5.8,261
2,tt0000005,6.2,2554
3,tt0000006,5.1,175
4,tt0000007,5.4,797


In [24]:
# Open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


## Display Info of each Dataframe

In [25]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82581 entries, 0 to 82580
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          82581 non-null  object 
 1   titleType       82581 non-null  object 
 2   primaryTitle    82581 non-null  object 
 3   originalTitle   82581 non-null  object 
 4   isAdult         82581 non-null  int64  
 5   startYear       82581 non-null  int64  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  82581 non-null  int64  
 8   genres          82581 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 5.7+ MB


In [26]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478707 entries, 0 to 478706
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         478707 non-null  object 
 1   averageRating  478707 non-null  float64
 2   numVotes       478707 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 11.0+ MB


In [27]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1392037 entries, 0 to 1392036
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1392037 non-null  object 
 1   ordering         1392037 non-null  int64  
 2   title            1392037 non-null  object 
 3   region           1392037 non-null  object 
 4   language         3754 non-null     object 
 5   types            967399 non-null   object 
 6   attributes       45507 non-null    object 
 7   isOriginalTitle  1390692 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 85.0+ MB
